In [26]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

# Import our input dataset
supply_chain_df = pd.read_csv('DataCoSupplyChainDataset.csv',encoding='ISO-8859-1')
supply_chain_df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [28]:
#Remove un-needed columns
supply_chain_clean_df = supply_chain_df.drop(columns=["Customer Email","Customer Fname","Category Id","Customer Lname","Customer Password","Customer City","Customer Id","Customer Street","Customer Zipcode",
                                                      "Latitude","Longitude","Department Id","Department Name","Order Customer Id","Order Id","Order Item Cardprod Id","Order Item Product Price","Order Item Discount",
                                                     "Product Card Id","Product Category Id","Product Description","Product Name","Product Price","Product Image","Order Zipcode"])
supply_chain_clean_df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Name,Customer Country,Customer Segment,...,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,Sporting Goods,Puerto Rico,Consumer,...,1,327.75,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,Sporting Goods,Puerto Rico,Consumer,...,1,327.75,311.359985,-249.089996,South Asia,Rajastán,PENDING,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,Sporting Goods,EE. UU.,Consumer,...,1,327.75,309.720001,-247.779999,South Asia,Rajastán,CLOSED,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,Sporting Goods,EE. UU.,Home Office,...,1,327.75,304.809998,22.860001,Oceania,Queensland,COMPLETE,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,Sporting Goods,Puerto Rico,Corporate,...,1,327.75,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,0,1/15/2018 11:24,Standard Class


In [27]:
data_types_df = pd.DataFrame(supply_chain_df.dtypes)
data_types_df

,0
Type,object
Days for shipping (real),int64
Days for shipment (scheduled),int64
Benefit per order,float64
Sales per customer,float64
Delivery Status,object
Late_delivery_risk,int64
Category Id,int64
Category Name,object
Customer City,object


In [29]:
data_types_df = pd.DataFrame(supply_chain_clean_df.dtypes)
data_types_df

,0
Type,object
Days for shipping (real),int64
Days for shipment (scheduled),int64
Benefit per order,float64
Sales per customer,float64
Delivery Status,object
Late_delivery_risk,int64
Category Name,object
Customer Country,object
Customer Segment,object


In [36]:
supply_chain_clean_df = supply_chain_clean_df.dropna(axis='columns', how='all')
supply_chain_clean_df

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Name,Customer Country,Customer Segment,...,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Product Status,shipping date (DateOrders),Shipping Mode,Shipping Delay (# of days)
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,Sporting Goods,Puerto Rico,Consumer,...,327.750000,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,0,2/3/2018 22:56,Standard Class,-1
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,Sporting Goods,Puerto Rico,Consumer,...,327.750000,311.359985,-249.089996,South Asia,Rajastán,PENDING,0,1/18/2018 12:27,Standard Class,1
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,Sporting Goods,EE. UU.,Consumer,...,327.750000,309.720001,-247.779999,South Asia,Rajastán,CLOSED,0,1/17/2018 12:06,Standard Class,0
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,Sporting Goods,EE. UU.,Home Office,...,327.750000,304.809998,22.860001,Oceania,Queensland,COMPLETE,0,1/16/2018 11:45,Standard Class,-1
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,Sporting Goods,Puerto Rico,Corporate,...,327.750000,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,0,1/15/2018 11:24,Standard Class,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,4,4,40.000000,399.980011,Shipping on time,0,Fishing,EE. UU.,Home Office,...,399.980011,399.980011,40.000000,Eastern Asia,Shanghái,CLOSED,0,1/20/2016 3:40,Standard Class,0
180515,DEBIT,3,2,-613.770019,395.980011,Late delivery,1,Fishing,EE. UU.,Corporate,...,399.980011,395.980011,-613.770019,Eastern Asia,Osaka,COMPLETE,0,1/19/2016 1:34,Second Class,1
180516,TRANSFER,5,4,141.110001,391.980011,Late delivery,1,Fishing,EE. UU.,Corporate,...,399.980011,391.980011,141.110001,Oceania,Australia del Sur,PENDING,0,1/20/2016 21:00,Standard Class,1
180517,PAYMENT,3,4,186.229996,387.980011,Advance shipping,0,Fishing,Puerto Rico,Consumer,...,399.980011,387.980011,186.229996,Oceania,Australia del Sur,PENDING_PAYMENT,0,1/18/2016 20:18,Standard Class,-1


In [37]:
# Remove Null columns
supply_chain_clean_df = supply_chain_clean_df.dropna()
supply_chain_clean_df

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Name,Customer Country,Customer Segment,...,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Product Status,shipping date (DateOrders),Shipping Mode,Shipping Delay (# of days)
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,Sporting Goods,Puerto Rico,Consumer,...,327.750000,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,0,2/3/2018 22:56,Standard Class,-1
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,Sporting Goods,Puerto Rico,Consumer,...,327.750000,311.359985,-249.089996,South Asia,Rajastán,PENDING,0,1/18/2018 12:27,Standard Class,1
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,Sporting Goods,EE. UU.,Consumer,...,327.750000,309.720001,-247.779999,South Asia,Rajastán,CLOSED,0,1/17/2018 12:06,Standard Class,0
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,Sporting Goods,EE. UU.,Home Office,...,327.750000,304.809998,22.860001,Oceania,Queensland,COMPLETE,0,1/16/2018 11:45,Standard Class,-1
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,Sporting Goods,Puerto Rico,Corporate,...,327.750000,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,0,1/15/2018 11:24,Standard Class,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,4,4,40.000000,399.980011,Shipping on time,0,Fishing,EE. UU.,Home Office,...,399.980011,399.980011,40.000000,Eastern Asia,Shanghái,CLOSED,0,1/20/2016 3:40,Standard Class,0
180515,DEBIT,3,2,-613.770019,395.980011,Late delivery,1,Fishing,EE. UU.,Corporate,...,399.980011,395.980011,-613.770019,Eastern Asia,Osaka,COMPLETE,0,1/19/2016 1:34,Second Class,1
180516,TRANSFER,5,4,141.110001,391.980011,Late delivery,1,Fishing,EE. UU.,Corporate,...,399.980011,391.980011,141.110001,Oceania,Australia del Sur,PENDING,0,1/20/2016 21:00,Standard Class,1
180517,PAYMENT,3,4,186.229996,387.980011,Advance shipping,0,Fishing,Puerto Rico,Consumer,...,399.980011,387.980011,186.229996,Oceania,Australia del Sur,PENDING_PAYMENT,0,1/18/2016 20:18,Standard Class,-1


In [38]:
# Add a column for number of days of shipping delays
supply_chain_clean_df["Shipping Delay (# of days)"] = supply_chain_clean_df["Days for shipping (real)"] - supply_chain_clean_df["Days for shipment (scheduled)"]
supply_chain_clean_df

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Name,Customer Country,Customer Segment,...,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Product Status,shipping date (DateOrders),Shipping Mode,Shipping Delay (# of days)
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,Sporting Goods,Puerto Rico,Consumer,...,327.750000,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,0,2/3/2018 22:56,Standard Class,-1
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,Sporting Goods,Puerto Rico,Consumer,...,327.750000,311.359985,-249.089996,South Asia,Rajastán,PENDING,0,1/18/2018 12:27,Standard Class,1
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,Sporting Goods,EE. UU.,Consumer,...,327.750000,309.720001,-247.779999,South Asia,Rajastán,CLOSED,0,1/17/2018 12:06,Standard Class,0
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,Sporting Goods,EE. UU.,Home Office,...,327.750000,304.809998,22.860001,Oceania,Queensland,COMPLETE,0,1/16/2018 11:45,Standard Class,-1
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,Sporting Goods,Puerto Rico,Corporate,...,327.750000,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,0,1/15/2018 11:24,Standard Class,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,4,4,40.000000,399.980011,Shipping on time,0,Fishing,EE. UU.,Home Office,...,399.980011,399.980011,40.000000,Eastern Asia,Shanghái,CLOSED,0,1/20/2016 3:40,Standard Class,0
180515,DEBIT,3,2,-613.770019,395.980011,Late delivery,1,Fishing,EE. UU.,Corporate,...,399.980011,395.980011,-613.770019,Eastern Asia,Osaka,COMPLETE,0,1/19/2016 1:34,Second Class,1
180516,TRANSFER,5,4,141.110001,391.980011,Late delivery,1,Fishing,EE. UU.,Corporate,...,399.980011,391.980011,141.110001,Oceania,Australia del Sur,PENDING,0,1/20/2016 21:00,Standard Class,1
180517,PAYMENT,3,4,186.229996,387.980011,Advance shipping,0,Fishing,Puerto Rico,Consumer,...,399.980011,387.980011,186.229996,Oceania,Australia del Sur,PENDING_PAYMENT,0,1/18/2016 20:18,Standard Class,-1


In [39]:
columns = supply_chain_clean_df.index
target = ["Late_delivery_risk"]

In [40]:
# Convert Late delivery risk
x = {1: 'sent late',0: 'sent on time'}  
# Change tarrget to difference betweenn days for shipping real and days for shipment scheduled
supply_chain_clean_df["Late_delivery_risk"] = supply_chain_clean_df["Late_delivery_risk"].replace(x)
supply_chain_clean_df

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Name,Customer Country,Customer Segment,...,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Product Status,shipping date (DateOrders),Shipping Mode,Shipping Delay (# of days)
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,sent on time,Sporting Goods,Puerto Rico,Consumer,...,327.750000,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,0,2/3/2018 22:56,Standard Class,-1
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,sent late,Sporting Goods,Puerto Rico,Consumer,...,327.750000,311.359985,-249.089996,South Asia,Rajastán,PENDING,0,1/18/2018 12:27,Standard Class,1
2,CASH,4,4,-247.779999,309.720001,Shipping on time,sent on time,Sporting Goods,EE. UU.,Consumer,...,327.750000,309.720001,-247.779999,South Asia,Rajastán,CLOSED,0,1/17/2018 12:06,Standard Class,0
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,sent on time,Sporting Goods,EE. UU.,Home Office,...,327.750000,304.809998,22.860001,Oceania,Queensland,COMPLETE,0,1/16/2018 11:45,Standard Class,-1
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,sent on time,Sporting Goods,Puerto Rico,Corporate,...,327.750000,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,0,1/15/2018 11:24,Standard Class,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,4,4,40.000000,399.980011,Shipping on time,sent on time,Fishing,EE. UU.,Home Office,...,399.980011,399.980011,40.000000,Eastern Asia,Shanghái,CLOSED,0,1/20/2016 3:40,Standard Class,0
180515,DEBIT,3,2,-613.770019,395.980011,Late delivery,sent late,Fishing,EE. UU.,Corporate,...,399.980011,395.980011,-613.770019,Eastern Asia,Osaka,COMPLETE,0,1/19/2016 1:34,Second Class,1
180516,TRANSFER,5,4,141.110001,391.980011,Late delivery,sent late,Fishing,EE. UU.,Corporate,...,399.980011,391.980011,141.110001,Oceania,Australia del Sur,PENDING,0,1/20/2016 21:00,Standard Class,1
180517,PAYMENT,3,4,186.229996,387.980011,Advance shipping,sent on time,Fishing,Puerto Rico,Consumer,...,399.980011,387.980011,186.229996,Oceania,Australia del Sur,PENDING_PAYMENT,0,1/18/2016 20:18,Standard Class,-1


In [42]:
# Create our features
X = supply_chain_clean_df.drop("Late_delivery_risk", axis=1)

X = pd.get_dummies(X)

# Create our target
y = supply_chain_clean_df["Late_delivery_risk"]

In [43]:
X.describe()

MemoryError: Unable to allocate 22.6 GiB for an array with shape (134453, 180519) and data type uint8

In [44]:
# Check the balance of our target values
y.value_counts()

sent late       98977
sent on time    81542
Name: Late_delivery_risk, dtype: int64

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

MemoryError: Unable to allocate 22.6 GiB for an array with shape (134453, 180519) and data type uint8

In [46]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

NameError: name 'X_train' is not defined

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)